In [34]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [35]:
import logging
from pathlib import Path

import altair as alt
import holoviews as hv
import hvplot.polars  # noqa
import numpy as np
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.eda import detrend_tonic_component
from src.features.scaling import scale_min_max
from src.features.utils import to_describe
from src.log_config import configure_logging
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
    plot_max_correlations_by_participant,
)

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [ ]:
db = DatabaseManager()
with db:
    df = db.get_table(
        "Merged_and_Labeled_Data",
        exclude_trials_with_measurement_problems=True,
    )

# Rename columns
df = df.rename(
    {
        "rating": "pain_rating",
    }
)

# Remove first 20 seconds
df = df.filter(col("normalized_timestamp") >= 20 * 1000)
df

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heart_rate,heart_rate_exploratory,heart_rate_smooth,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_r_exploratory,pupil_l_exploratory,pupil_mean,pupil_mean_exploratory,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
1,1,1,314198.9762,0.463265,0.87188,0.743199,0.743294,-0.000095,1375.838609,-1.67065,59.967043,59.961244,59.245111,4.577485,4.825832,4.933592,4.448417,4.933559,4.448429,4.691004,4.690994,0.000985,0.000446,0.001714,0.000036,0.000103,20000.0,396,1,0,0,1,0,0,1,0
1,1,1,314298.9762,0.463265,0.824514,0.74323,0.743297,-0.000067,1349.613188,-0.775775,60.322349,60.332773,59.619581,4.590687,4.831406,4.934665,4.446768,4.934707,4.446755,4.690717,4.690731,0.000957,0.000443,0.001568,0.000032,0.000097,20100.0,396,1,0,0,1,0,0,1,0
1,1,1,314398.9762,0.463265,0.815,0.743276,0.7433,-0.000023,1325.324829,-0.88756,60.730059,60.725636,60.01789,4.593487,4.862389,4.930151,4.476284,4.930121,4.476294,4.703218,4.703207,0.000931,0.000441,0.001442,0.00003,0.000094,20200.0,396,1,0,0,1,0,0,1,0
1,1,1,314498.9762,0.463265,0.815,0.743297,0.743303,-0.000006,1297.487842,-1.209588,61.187669,61.149882,60.464061,4.555876,4.885492,4.907607,4.518567,4.907623,4.518566,4.713087,4.713094,0.000912,0.000442,0.001316,0.000027,0.00009,20300.0,396,1,0,0,1,0,0,1,0
1,1,1,314598.9762,0.463265,0.815,0.743265,0.743306,-0.000041,1291.534487,-0.726068,61.595507,61.587131,60.917462,4.497064,4.882886,4.888406,4.528388,4.888402,4.528379,4.708397,4.708391,0.000885,0.000436,0.001194,0.000024,0.000087,20400.0,396,1,0,0,1,0,0,1,0
1,1,1,314698.9762,0.463265,0.815,0.743194,0.74331,-0.000117,1371.283878,-1.318697,62.031792,62.052729,61.375585,4.464745,4.862969,4.882415,4.528865,4.882412,4.528885,4.70564,4.705649,0.000879,0.000433,0.00106,0.000021,0.000082,20500.0,396,1,0,0,1,0,0,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
516,12,50,2.6956e6,0.031312,0.0,26.688608,26.716225,0.003169,1323.813266,-5.036073,60.344651,59.279672,60.345642,5.166699,4.766597,5.029708,5.52614,5.029708,5.526104,5.277924,5.277906,0.000286,0.001043,0.010694,0.000266,0.002656,179500.0,806,1,2580,1548,0,0,0,0,0
516,12,50,2.6957e6,0.030874,0.0,26.68499,26.716212,0.000389,1244.871959,44.337204,59.578481,58.673362,60.237089,5.026438,4.683481,5.016028,5.473471,5.016028,5.473472,5.244749,5.24475,0.000316,0.001028,0.010443,0.000273,0.002769,179600.0,806,1,2580,1548,0,0,0,0,0


In [39]:
facial_features = [
    "brow_furrow",
    "brow_furrow",
    "cheek_raise",
    "mouth_open",
    "upper_lip_raise",
    "nose_wrinkle",
]


In [40]:
df.filter(participant_id=35, trial_number=7)

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heart_rate,heart_rate_exploratory,heart_rate_smooth,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_r_exploratory,pupil_l_exploratory,pupil_mean,pupil_mean_exploratory,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
365,7,35,1.4930e6,0.499774,0.150475,21.272144,21.64738,-0.375236,1436.4905,-0.947256,75.135777,75.115871,75.014853,3.803774,3.482999,3.529268,3.811099,3.529231,3.811126,3.670184,3.670179,0.204629,0.000077,0.000902,0.000355,0.000886,20000.0,133,1,1821,1093,0,0,0,0,0
365,7,35,1.4931e6,0.492438,0.139078,21.262946,21.635876,-0.37293,1406.646967,-0.951448,75.120867,75.212463,75.062575,3.766977,3.46357,3.528845,3.815561,3.528856,3.815534,3.672203,3.672195,0.197543,0.000078,0.00092,0.000365,0.000945,20100.0,133,1,1821,1093,0,0,0,0,0
365,7,35,1.4932e6,0.485243,0.132785,21.248235,21.624447,-0.376212,1391.406818,-1.708674,75.143875,75.329956,75.087998,3.749578,3.477617,3.528683,3.815048,3.528667,3.815049,3.671865,3.671858,0.19157,0.000082,0.000853,0.000377,0.000969,20200.0,133,1,1821,1093,0,0,0,0,0
365,7,35,1.4933e6,0.477816,0.125222,21.226279,21.612996,-0.386716,1370.632261,2.182886,75.099339,75.481393,75.078618,3.74758,3.495737,3.530116,3.815158,3.530109,3.815156,3.672637,3.672633,0.183972,0.000084,0.000781,0.000375,0.000946,20300.0,133,1,1821,1093,0,0,0,0,0
365,7,35,1.4934e6,0.470522,0.120155,21.216717,21.601409,-0.384692,1479.129824,47.062661,75.499511,75.666963,75.164001,3.750548,3.504035,3.522115,3.81622,3.52213,3.816203,3.669167,3.669167,0.186087,0.000084,0.000747,0.000366,0.00092,20400.0,133,1,1821,1093,0,0,0,0,0
365,7,35,1.4935e6,0.463231,0.115158,21.210887,21.5911,-0.380213,1586.916431,27.511407,76.010641,75.836043,75.458397,3.749731,3.509338,3.509267,3.798878,3.509241,3.798894,3.654072,3.654068,0.190849,0.000083,0.000693,0.00036,0.000953,20500.0,133,1,1821,1093,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
365,7,35,1.6525e6,0.158931,0.599919,18.158782,18.153983,0.09695,1292.326488,8.081089,74.147744,73.599215,74.117298,3.629625,3.230731,3.169647,3.524443,3.169478,3.524704,3.347045,3.347091,0.164506,0.000059,0.000177,0.000083,0.000641,179500.0,133,1,1825,0,0,0,0,0,0
365,7,35,1.6526e6,0.157472,0.546606,18.138025,18.154059,0.085615,1316.479733,47.070152,73.35728,73.281258,73.930294,3.699183,3.286455,3.171115,3.529308,3.169713,3.525019,3.350211,3.347366,0.167316,0.00006,0.000178,0.000081,0.000643,179600.0,133,1,1825,0,0,0,0,0,0


# Temperature

In [41]:
COLORS = {
    "temperature_brow_furrow_corr": "red",
    "temperature_cheek_raise_corr": "#2ca02c",
    "temperature_mouth_open_corr": "#d62728",
    "temperature_upper_lip_raise_corr": "#9467bd",
    "temperature_nose_wrinkle_corr": "#ff7f0e",
}

In [42]:
plots = {}

for feature in facial_features:
    col1, col2 = "temperature", feature

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corr_by_participant = aggregate_correlations_fisher_z(
        corr_by_trial, col1, col2, "participant_id", include_ci=True
    )
    plot_correlations_by_trial(corr_by_trial, col1, col2)
    plots[feature] = plot_correlations_by_participant(
        corr_by_participant, col1, col2, with_config=False, color_map=COLORS
    )

(
    plots["brow_furrow"]
    + plots["cheek_raise"]
    + plots["mouth_open"]
    + plots["upper_lip_raise"]
    + plots["nose_wrinkle"]
)

17:55:18 | DEBUG   | correlations | Removing NaN correlations
17:55:18 | DEBUG   | correlations | Removing NaN correlations
17:55:18 | DEBUG   | correlations | Removing NaN correlations


alt.LayerChart(...)

# Rating

In [46]:
COLORS = {
    "pain_rating_brow_furrow_corr": "red",
    "pain_rating_cheek_raise_corr": "#2ca02c",
    "pain_rating_mouth_open_corr": "#d62728",
    "pain_rating_upper_lip_raise_corr": "#9467bd",
    "pain_rating_nose_wrinkle_corr": "#ff7f0e",
}

In [47]:
plots = {}

for feature in facial_features:
    col1, col2 = "pain_rating", feature

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corr_by_participant = aggregate_correlations_fisher_z(
        corr_by_trial, col1, col2, "participant_id", include_ci=True
    )
    plot_correlations_by_trial(corr_by_trial, col1, col2)
    plots[feature] = plot_correlations_by_participant(
        corr_by_participant, col1, col2, with_config=False, color_map=COLORS
    )

(
    plots["brow_furrow"]
    + plots["cheek_raise"]
    + plots["mouth_open"]
    + plots["upper_lip_raise"]
    + plots["nose_wrinkle"]
)

17:55:50 | DEBUG   | correlations | Removing NaN correlations
17:55:50 | DEBUG   | correlations | Removing NaN correlations
17:55:50 | DEBUG   | correlations | Removing NaN correlations


alt.LayerChart(...)